In [1]:
import os
from zeroae.rocksdb import c as crocksdb

In [2]:
DBPath="/tmp/rocksdb_simple_example"
DBBackupPath="/tmp/rocksdb_simple_example_backup"

### Optimize RocksDB.
The easiest way to get RocksDB to perform well (use all your cores)

In [3]:
options = crocksdb.options.create()
crocksdb.options.increase_parallelism(options,os.cpu_count())
crocksdb.options.optimize_level_style_compaction(options,0)
crocksdb.options.set_create_if_missing(options, 1)

### Open the DB

In [4]:
db, err = crocksdb.open(options, DBPath)
assert err is None

### Open Backup Engine
  We will use it for backing up our database.

In [5]:
be, err = crocksdb.backup_engine.open(options, DBBackupPath);
assert err is None

### Put key-value

In [6]:
writeoptions = crocksdb.writeoptions.create()
key = "key"
err = crocksdb.put(db, writeoptions, key, "value")
assert err is None
crocksdb.writeoptions.destroy(writeoptions)

### Get value

In [7]:
%%inactive
readoptions = crocksdb.readoptions.create()
rv, err = crocksdb.get(db, readoptions, "key")
#rv, err = c.db.get(db, readoptions, key, strlen(key), &len);
assert err is None
assert rv == "value"
crocksdb.readoptions.destroy(readoptions)

UsageError: Cell magic `%%inactive` not found.


### Create a backup in DBBackupPath,

In [8]:
err = crocksdb.backup_engine.create_new_backup(be, db)
assert err is None
crocksdb.close(db)

### Restore DB from latest backup

In [9]:
restore_options = crocksdb.restore_options.create()
err = crocksdb.backup_engine.restore_db_from_latest_backup(
    be, DBPath, DBPath, restore_options
)
assert err is None
crocksdb.restore_options.destroy(restore_options)

### Reopen the restored database

In [10]:
db, err = crocksdb.open(options, DBPath)
assert err is None

### Cleanup

In [11]:
crocksdb.options.destroy(options)
crocksdb.backup_engine.close(be)
crocksdb.close(db)